# Creating BDT input pkls from input ntuples

as well as the final pkls for use in the main analysis.

The purpose of this notebooke was to see if the updated NuMI flux ntuples could be ported into the main analysis body, and the effect this would have.

In [41]:
# So that the notebook us updated to reflect changes in the .py file:
%load_ext autoreload
%autoreload 2
from create_pkls import * # .py file used to define most functions
# Note: Importing uproot3 as uproot so that we can flatten more easily


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
# Parameters to set so that files are saved/imported correctly
run = 'Run4b' 
sample = 'Ext' # 'Data', 'Signal', 'Overlay', 'Dirt', 'Ext'
bkg_dict = {'Overlay': 'o',
            'Dirt': 'd',
            'Ext': 'e',}

load_vars = Main_variables.copy()

if sample == 'Overlay' or sample == 'Dirt':
    load_vars += ["ppfx_cv"]

# Defining the pkl save path
loc_BDT_input = '/exp/uboone/data/users/jbateman/workdir/HPS_uboone_analysis/Run_245_ana/BDT_input_pkl/'

loc_ntuple = '/exp/uboone/data/users/jbateman/workdir/HPS_uboone_analysis/Run_245_ana/'
ntuple_name = 'numi_pelee_ntuple_beam_off_run4b_rhc_ana.root'

root_tree = uproot.open(loc_ntuple + ntuple_name)['nuselection/NeutrinoSelectionFilter']

# Option to add extra variables to main/additional arrays, based on the sample type.
df_main_vars = root_tree.pandas.df(load_vars,flatten=True) 
print('Created main dataframe.')

additional_variables = mc_variables+weights_variables+truth_variables+tune_variables
if sample=='Overlay' or sample == 'Dirt':
    print('Adding overlay variables to the dataframe.')
    df_jagged_vars = root_tree.pandas.df(additional_variables, flatten=False)
    df_main_vars = pd.merge(df_main_vars, df_jagged_vars, on='entry', how='inner')
    df_main_vars = weightTuneBool(df_main_vars)

    


The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
Created main dataframe.


## Creating the 1 & 2 Shower samples

In [83]:
df_temp = df_main_vars.copy(deep=True)
df_temp_1shr = preselection_1shr(df_temp, print_bool=True)
df_temp_1shr = get_1shr_variable (df_temp_1shr.copy())

if sample =='Signal':
    df_temp_1shr = merge_true_theta_bkg(df_temp_1shr.copy(), 'Run4a')
elif sample in bkg_dict:
    df_temp_1shr = tag_bkg_category(df_temp_1shr.copy(), bkg_dict[sample]) # o: overlay, d: dirt, e: ext

df_temp_2shr = preselection_2shr(df_temp, print_bool=True)
df_temp_2shr = leading_shr(df_temp_2shr.copy())
df_temp_2shr = get_2shr_variable(df_temp_2shr.copy())

if sample =='Signal':
    df_temp_2shr = merge_true_theta_bkg(df_temp_2shr.copy(), 'Run4a')
elif sample in bkg_dict:
    df_temp_2shr = tag_bkg_category(df_temp_2shr.copy(), bkg_dict[sample]) # o: overlay, d: dirt, e: ext




Before slice ID cut
267959
Before FV cut
267959
After FV cut
136752
After nu_purity_from_pfp cut
136752
After CRT cut
52847
After trk_score cut
11893
After shr_pfp_id_v cut
10251
After n_showers_contained_MCStool cut
6432
After Counts==1 cut
6046
After FV cut
136752
After nu_purity_from_pfp cut
136752
After FV cut
136752
After CRT cut
52847
After trk_score cut
11893
After shr_pfp_id_v cut
10251
After n_showers_contained_MCStool cut
2071
After Counts==2 cut
1922
960
962
959


/exp/uboone/app/users/jbateman/workdir/ALPs_and_HPS/HPS_ALP_Analysis/BDT_pkl_gen/create_pkls.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Counts'] = df.groupby(['entry'])['temp_ID'].transform('count')
/exp/uboone/app/users/jbateman/workdir/ALPs_and_HPS/HPS_ALP_Analysis/BDT_pkl_gen/create_pkls.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df = df.query('Counts==1')
/exp/uboone/app/users/jbateman/workdir/ALPs_and_HPS/HPS_ALP_Analysis/BDT_pkl_gen/create_pkls.py:217: SettingWithCopyWarning:

960
df
1922
df_temp_temp
4
960
961
960
961


## Getting a BDT prediction

In [84]:
BDT_path = '/exp/uboone/data/users/jbateman/workdir/aadi/jupyter_koto/Booster2sens/'

# /Run1_150_MeV_NEW/BDT_models/BDT_Model_Top_14_features_Run1_150_MeVKDIF_1shr_Final_Main_BDT'
# AllVar_Selected_Run3_NuMI_Overlay_1shr_Updated_Flux
bdt_run = 'Run3'

mass_points = ['100_MeV', '125_MeV', '130_MeV', '135_MeV','140_MeV', '145_MeV', '150_MeV', '200_MeV']
kaons = ['KDIF', 'KDAR']
shrs = ['1shr','2shr']

df_arr = [df_temp_1shr, df_temp_2shr]

df_pred_dict = {}

for mass in mass_points:
    for k in kaons:
        i = 0
        for shr in shrs:
            BDT_name = f"/{bdt_run}_{mass}_NEW/BDT_models/BDT_Model_Top_14_features_{bdt_run}_{mass}{k}_{shr}_Final_Main_BDT"
            booster = joblib.load(BDT_path + BDT_name)
            df_temp = df_arr[i].copy()
            print(f"{bdt_run}_{mass}_{k}_{shr}")
            print(len(df_temp['run_ls']))
            cols_for_booster = booster.feature_names
            untrnd_df = df_temp[cols_for_booster].copy()
            feature_names = untrnd_df.columns.to_list()
            untrnd_final = xgboost.DMatrix(data=untrnd_df[feature_names],missing=-999.0,feature_names=feature_names)
            pred = booster.predict(untrnd_final)
            print('Calculated predictions...')
            
            df_temp['pred'] = pred
            
            cols_to_use  = ['weightsPPFX_ls', 'weightsGenie_ls', 'weightsReint_ls']
            if shr == '2shr' and (sample == 'Overlay' or sample == 'Dirt'): 
                cols_to_drop = ['weightsPPFX_sls', 'weightsGenie_sls', 'weightsReint_sls']
                df_temp = df_temp.drop(cols_to_drop, axis=1)
            df_temp = df_temp.rename(columns={'weightsPPFX_ls': 'weightsPPFX',
                            'weightsGenie_ls': 'weightsGenie',
                            'weightsReint_ls': 'weightsReint',
                            # Renaming these params for parity with old pkl files
                            'mc_px_ls': 'mc_primary_px_ls',
                            'mc_py_ls': 'mc_primary_py_ls',
                            'mc_pz_ls': 'mc_primary_pz_ls',
                            'mc_px_sls': 'mc_primary_px_sls',
                            'mc_py_sls': 'mc_primary_py_sls',
                            'mc_pz_sls': 'mc_primary_pz_sls',
                            'mc_pdg_ls': 'mc_primary_pdg_ls',
                            'mc_pdg_sls': 'mc_primary_pdg_sls'
                            })
            
            df_temp['is_trained'] = 0
            df_temp['run'] = df_temp['run_ls']
            df_temp['sub'] = df_temp['sub_ls']
            df_temp['evt'] = df_temp['evt_ls']
            df_temp['nslice'] = df_temp['nslice_ls']
            print('Fixed column labelling...')
            
            # Calculating these parameters early, to save so much time on importing these pkls later
            if sample == 'Overlay':
                df_temp['thsnd_weight_genie'] = (['yes' if all(a == 1000 for a in i) else 'no' for i in df_temp['weightsGenie']])
                df_temp['thsnd_weight_ppfx'] = (['yes' if all(a == 1000 for a in i) else 'no' for i in df_temp['weightsPPFX']])
                df_temp['thsnd_weight_reint'] = (['yes' if all(a == 1000 for a in i) else 'no' for i in df_temp['weightsReint']])
                
                df_temp['inf_weight_genie'] = (['yes' if any(a>65534 for a in i) else 'no' for i in df_temp['weightsGenie']])
                df_temp['inf_weight_ppfx'] = (['yes' if any(a>65534 for a in i) else 'no' for i in df_temp['weightsPPFX']])
                df_temp['inf_weight_reint'] = (['yes' if any(a>65534 for a in i) else 'no' for i in df_temp['weightsReint']])

                df_temp['ones_weight_genie'] = (['yes' if all(a == 1 for a in i) else 'no' for i in df_temp['weightsGenie']])
                df_temp['ones_weight_ppfx'] = (['yes' if all(a == 1 for a in i) else 'no' for i in df_temp['weightsPPFX']])
                df_temp['ones_weight_reint'] = (['yes' if all(a == 1 for a in i) else 'no' for i in df_temp['weightsReint']])
                
                df_temp['zero_weight_genie'] = (['yes' if all(a == 0 for a in i) else 'no' for i in df_temp['weightsGenie']])
                df_temp['zero_weight_ppfx'] = (['yes' if all(a == 0 for a in i) else 'no' for i in df_temp['weightsPPFX']])
                df_temp['zero_weight_reint'] = (['yes' if all(a == 0 for a in i) else 'no' for i in df_temp['weightsReint']])
                print('Calculated PPFX, Genie, Reint flags...')
            df_pred_dict[f"{run}_{mass}_{k}_{shr}"] = df_temp

            i+=1


            

/exp/uboone/app/users/jbateman/conda/envs/HPS_ana/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:12:05] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


Run3_100_MeV_KDIF_1shr
6046
Calculated predictions...
Fixed column labelling...


/exp/uboone/app/users/jbateman/conda/envs/HPS_ana/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:12:06] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


Run3_100_MeV_KDIF_2shr
963
Calculated predictions...
Fixed column labelling...


/exp/uboone/app/users/jbateman/conda/envs/HPS_ana/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:12:06] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


...

Run3_200_MeV_KDIF_2shr
963
Calculated predictions...
Fixed column labelling...


/exp/uboone/app/users/jbateman/conda/envs/HPS_ana/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:12:19] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


Run3_200_MeV_KDAR_1shr
6046
Calculated predictions...
Fixed column labelling...


/exp/uboone/app/users/jbateman/conda/envs/HPS_ana/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:12:19] WARNING: /workspace/src/learner.cc:872: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


Run3_200_MeV_KDAR_2shr
963
Calculated predictions...
Fixed column labelling...


Removing the ppfx_cv_x/y that appears during dataframe merging

In [ ]:
for mass in mass_points:
    mass_int = int(mass.split('_')[0]) 
    for k in kaons:
        i = 0
        for shr in shrs:
            sample_name = sample
            if sample == 'Data':
                sample_name = 'Opendata'
            df_temp = df_pred_dict[f"{run}_{mass}_{k}_{shr}"].copy()
            if 'ppfx_cv_x_ls' in df_temp.columns and 'ppfx_cv_y_ls' in df_temp.columns:
                if all(df_temp['ppfx_cv_x_ls'] == df_temp['ppfx_cv_y_ls']):
                    print('PPFX CV is the same for both slices, so removing the second slice.')
                    df_temp.drop(columns=['ppfx_cv_y_ls'], inplace=True)
                    df_temp.rename(columns={'ppfx_cv_x_ls': 'ppfx_cv_ls'}, inplace=True)
            if 'ppfx_cv_x_sls' in df_temp.columns and 'ppfx_cv_y_sls' in df_temp.columns:
                if all(df_temp['ppfx_cv_x_sls'] == df_temp['ppfx_cv_y_sls']):
                    print('PPFX CV is the same for both slices, so removing the second slice.')
                    df_temp.drop(columns=['ppfx_cv_y_sls'], inplace=True)
                    df_temp.rename(columns={'ppfx_cv_x_sls': 'ppfx_cv_sls'}, inplace=True)

            df_temp.to_pickle(loc_BDT_input + f"AllVar_Selected_{run}_NuMI_{sample_name}_{k}_{mass_int}_{shr}_PPFX_pred_NEW.pkl")
            print(f"Saved AllVar_Selected_{run}_NuMI_{sample_name}_{k}_{mass_int}_{shr}_PPFX_pred_NEW.pkl")

Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_100_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_100_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_100_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_100_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_125_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_125_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_125_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_125_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_130_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_130_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_130_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_130_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_135_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_135_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_E

Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_140_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_145_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_145_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_145_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_145_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_150_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_150_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_150_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_150_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_200_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDIF_200_2shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_200_1shr_PPFX_pred_NEW.pkl
Saved AllVar_Selected_Run4b_NuMI_Ext_KDAR_200_2shr_PPFX_pred_NEW.pkl


## Taking a look at the detsys samples...
How do we need our detsys samples to be formatted?

In [ ]:
ovl_detsys_labels = [
        "CV",
        "LYAttenuation",
        "LYDown",
        "LYRayleigh",
        "Recomb2",
        "SCE",
        "WireModThetaXZ",
        "WireModThetaYZ",
        "WireModdEdX",
        "WireModX",
        "WireModYZ",
            ]

In [ ]:
root_dir = "/exp/uboone/data/users/jbateman/workdir/HPS_uboone_analysis/"
detsys_mass = '150' 
detsys_bkg_input_dir = root_dir+'/Final_detector_systematics_BACKGROUND/'+detsys_mass+'_MeV/'
fhc_rhc_str = 'fhc' # 'fhc' or 'RHC'
KDIF_KDAR_str_lower = 'kdif' # 'kdif' or 'kdar'
nshr = '1shr' # '1shr' or '2shr'
df_ovl_detsys = pd.read_pickle(detsys_bkg_input_dir + "bkg_" + fhc_rhc_str + '_' + detsys_mass + '_MeV_' + nshr + '_' + KDIF_KDAR_str_lower +'.pkl')